# Instagram Scraping

Codi que va a les pàgina d'Instagram d'un restaurant i descarrega un diccionari de les fotos que allà apareixen amb el seu text, tags i url de la propia imatge.

hem emprat Selenium per aquest Scraping, fent scroll down per tal d'augmentar el nombre d'imatges sempre que sigui possible.

In [1]:
# per provar: 1342635 té més de 21 (surt load more), 259212659 té menys de 21 (no surt load more).
# per provar: 612111278965457 no té cap imatge

import os
import re
import time
import pickle
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
#  POSAR EL PATH AL chromedriver ( http://chromedriver.storage.googleapis.com/index.html?path=2.22/ )
path_to_chromedriver = '/Users/AlbertIribarne/Dropbox/UB/bigdata/instagram/chromedriver'

In [3]:
#  FUNCIONS PER GUARDAR I RECUPERAR DICCIONARIS EN FORMAT pickle (AIXÍ NO CAL TORNAR A CONSTRUIR-LO)

def save_obj(obj, name ):
    with open(''+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
def restaurant_insta_scraping_location(number):
    url='https://www.instagram.com/explore/locations/'+number
    browser.get(url)
    try:
        browser.find_element_by_class_name('_oidfu').click()
        more_than_21 = 1
    except NoSuchElementException:
        more_than_21 = 0
    if more_than_21 == 1:
        for i in range(20):
            time.sleep(1.5)
            browser.execute_script("window.scrollTo(0,document.body.scrollHeight);") #Scroll down 
        time.sleep(1)
        html=browser.page_source
        try:
            pictable = browser.find_element_by_xpath('/html/body/span/section/main/article/div[2]/div[1]')
            is_there_pictable = 1
        except NoSuchElementException:
            is_there_pictable = 0
        if is_there_pictable == 1:
            rest_info = []
            for element in pictable.find_elements_by_css_selector("a[class='_8mlbc _vbtk2 _t5r8b']"):
                img = element.find_element_by_class_name('_icyx7')
                dic = {}
                #element.get_attribute('href').replace("https://www.instagram.com/p/","").replace('/?tagged='+tag,"")
                #dic['tagsearch']= tag
                dic['text'] = img.get_attribute('alt')
                dic['tags'] = [t.replace("#","") for t in re.findall(r'#\w+', dic['text'])]
                dic['url'] = img.get_attribute('src').encode('ascii')
                rest_info.append(dic)
        else:
            rest_info = 0
            
    else: 
        html=browser.page_source
        try:
            pictable = browser.find_element_by_xpath('/html/body/span/section/main/article/div[2]/div[1]')
            is_there_pictable = 1
        except:
            is_there_pictable = 0
        if is_there_pictable == 1:
            rest_info = []
            for element in pictable.find_elements_by_css_selector("a[class='_8mlbc _vbtk2 _t5r8b']"):
                img = element.find_element_by_class_name('_icyx7')
                dic = {}
                dic['text'] = img.get_attribute('alt')
                dic['tags'] = [t.replace("#","") for t in re.findall(r'#\w+', dic['text'])]
                dic['url'] = img.get_attribute('src').encode('ascii')
                rest_info.append(dic)
        else:
            rest_info = 0
    return rest_info

In [5]:
#  Llista de diccionaris, un per restaurant.
#  Keys: NOMBRE, LOCATION_CODE, ESPECIALIDAD, DISTRITO, COORDENADAS, CIUDAD, CALLE
final_rest_list = pickle.load(open('final_rest_list.pkl', 'rb'))

In [69]:
#  Ara, a final_rest_list, se li adjunta a cada diccionari un item amb key IMAGES.
#  El value és una llista de diccionaris, un diccionari per imatge.
#  Els ítems d'aquest nou diccionari tenen keys: url, text, tags.

browser = webdriver.Chrome(path_to_chromedriver)

for item in final_rest_list:
    if final_rest_list.index(item) > 0:
        print (alberto_contador,item['NOMBRE'])
        to_append = []
        for code in item['LOCATION_CODE']:
            hola = restaurant_insta_scraping_location(code)
            print hola
            if hola != 0:
                to_append = to_append + hola
                item['IMAGES'] = to_append
        
browser.quit()

(0, 'Bar Restaurant Esterri')
[{'url': 'https://scontent-mad1-1.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/c31.0.1017.1017/13391328_1207600569273911_1720394920_n.jpg?ig_cache_key=MTI2OTQ5MzY5ODY5MzM2ODYxMg%3D%3D.2.c', 'text': u'#cafe #s\xf3lo #sola #cafeina #gogogo', 'tags': [u'cafe', u's', u'sola', u'cafeina', u'gogogo']}, {'url': 'https://scontent-mad1-1.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/10601666_1671372379817982_1564167151_n.jpg?ig_cache_key=MTE1OTY5MzgzODE1MDAyNjI2Mg%3D%3D.2', 'text': u'Oh Virtue! Where are the modest souls?', 'tags': []}, {'url': 'https://scontent-mad1-1.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/c194.0.692.692/11881570_443986505786655_1232354487_n.jpg?ig_cache_key=MTExMjUwNjMwNTAwMzM2ODE2Nw%3D%3D.2.c', 'text': u'#mojitotime #bellesones @aranzazugl93 @mar_og9', 'tags': [u'mojitotime', u'bellesones']}, {'url': 'https://scontent-mad1-1.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/c0.135.1080.1080/11909957_1166915779988828_225103213_n.j

In [79]:
save_obj(final_rest_list,'rest_list')